In [ ]:
import pandas as pd
import geopandas as gpd
import janitor as jn
from criminologia_cdmx.etl import *
from criminologia_cdmx.modelos import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
datos = get_carpetas_desde_archivo("datos/descargas/carpetas_fiscalia.csv")
datos = agrega_ids_espaciales(datos)
fecha_inicio = "01-01-2019"
fecha_fin = "01-01-2022"
columna_y = 'delito'
valores_y = ['ROBO A CASA HABITACION SIN VIOLENCIA', 'ROBO A CASA HABITACION CON VIOLENCIA']
fecha_inicio = pd.to_datetime(fecha_inicio, dayfirst=True)
fecha_fin = pd.to_datetime(fecha_fin, dayfirst=True)
datos = datos.loc[datos['fecha_hechos'].between(fecha_inicio, fecha_fin)]
datos = datos.loc[datos[columna_y].isin(valores_y)]

In [ ]:
st = (datos
      .groupby(['colonia_cve', 'delito'])
      .resample(rule='Y', on='fecha_hechos').size())
st

colonia_cve  delito                                fecha_hechos
1.0          ROBO A CASA HABITACION SIN VIOLENCIA  2019-12-31      1
                                                   2020-12-31      1
2.0          ROBO A CASA HABITACION CON VIOLENCIA  2019-12-31      1
                                                   2020-12-31      0
                                                   2021-12-31      1
                                                                  ..
1813.0       ROBO A CASA HABITACION CON VIOLENCIA  2020-12-31      1
                                                   2021-12-31      1
             ROBO A CASA HABITACION SIN VIOLENCIA  2019-12-31      3
                                                   2020-12-31      2
1814.0       ROBO A CASA HABITACION SIN VIOLENCIA  2019-12-31      4
Length: 5062, dtype: int64

In [ ]:
fechas_completas = pd.date_range(fecha_inicio, fecha_fin, freq='Y')

In [ ]:
st.reset_index()

,colonia_cve,delito,fecha_hechos,0
0,1.0,ROBO A CASA HABITACION SIN VIOLENCIA,2019-12-31,1
1,1.0,ROBO A CASA HABITACION SIN VIOLENCIA,2020-12-31,1
2,2.0,ROBO A CASA HABITACION CON VIOLENCIA,2019-12-31,1
3,2.0,ROBO A CASA HABITACION CON VIOLENCIA,2020-12-31,0
4,2.0,ROBO A CASA HABITACION CON VIOLENCIA,2021-12-31,1
...,...,...,...,...
5057,1813.0,ROBO A CASA HABITACION CON VIOLENCIA,2020-12-31,1
5058,1813.0,ROBO A CASA HABITACION CON VIOLENCIA,2021-12-31,1
5059,1813.0,ROBO A CASA HABITACION SIN VIOLENCIA,2019-12-31,3
5060,1813.0,ROBO A CASA HABITACION SIN VIOLENCIA,2020-12-31,2


In [ ]:
(st
 .reset_index()
 .complete({'fecha_hechos':fechas_completas, 'delito':valores_y}, by=['colonia_cve', 'delito'])
 .set_index(['colonia_cve', 'fecha_hechos']))

ValueError: 'delito' is both an index level and a column label, which is ambiguous.

In [ ]:
(st
 .reset_index()
 .complete( {'delito':valores_y}, by=['colonia_cve', 'fecha_hechos'])
 .set_index(['colonia_cve', 'fecha_hechos'])
 )

delito    0
colonia_cve fecha_hechos                                           
1.0         2019-12-31    ROBO A CASA HABITACION SIN VIOLENCIA  1.0
            2020-12-31    ROBO A CASA HABITACION SIN VIOLENCIA  1.0
2.0         2019-12-31    ROBO A CASA HABITACION CON VIOLENCIA  1.0
            2020-12-31    ROBO A CASA HABITACION CON VIOLENCIA  0.0
            2021-12-31    ROBO A CASA HABITACION CON VIOLENCIA  1.0
...                                                        ...  ...
1807.0      2019-12-31    ROBO A CASA HABITACION CON VIOLENCIA  NaN
1809.0      2021-12-31    ROBO A CASA HABITACION CON VIOLENCIA  NaN
1811.0      2021-12-31    ROBO A CASA HABITACION CON VIOLENCIA  NaN
1813.0      2021-12-31    ROBO A CASA HABITACION SIN VIOLENCIA  NaN
1814.0      2019-12-31    ROBO A CASA HABITACION CON VIOLENCIA  NaN

[7754 rows x 2 columns]